In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.plots import *
from torchtext.data import Field
from fastai.lm_rnn import seq2seq_reg
from quicknlp import SpacyTokenizer, print_batch, HierarchicalModelData
from pathlib import Path

In [3]:
!ls dataset/dialogue/

data  train  validation


Data found from https://www.kaggle.com/rtatman/ubuntu-dialogue-corpus 

In [4]:
path = Path("./dataset/dialogue")

In [5]:
ubuntu_data = pd.read_csv(path / "train/dialogueText.csv")

In [6]:
ubuntu_data.shape

(1038324, 6)

In [7]:
ubuntu_data['dialogueID'].nunique()
sampled_dialogues = set(ubuntu_data['dialogueID'].sample(1000).tolist())

# train_data = ubuntu_data.loc[ubuntu_data['dialogueID'] in sampled_dialogues.tolist()]

In [8]:
train_df = ubuntu_data.loc[ubuntu_data['dialogueID'].apply(lambda x: x in sampled_dialogues)]

In [9]:
ubuntu_data.columns

Index(['folder', 'dialogueID', 'date', 'from', 'to', 'text'], dtype='object')

In [10]:
from torchtext.data import field
from torchtext.data import Example

In [11]:
f = Field(tokenize="spacy", lower="True")

In [12]:
#columns to use when constructing the conversations
cols = {"text_col":"text", "batch_col":"dialogueID", "sort_col":"date", "role_col":"from"}

In [15]:
%pdb

Automatic pdb calling has been turned OFF


In [13]:
model_data = HierarchicalModelData.from_dataframes(path=path,
                                                   train_df=train_df,
                                                   val_df=train_df,
                                                   text_field=f,
                                                  **cols
                                                  )

100%|██████████| 999/999 [00:01<00:00, 526.49it/s]


In [28]:
dl =iter(model_data.trn_dl)

StopIteration: 